In [ ]:
# 문서 로딩
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


print("Loading document...")
loader = TextLoader("input/How_to_invest_money.txt", encoding="utf-8")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100
)

print("Splitting document into chunks...")
chunks = splitter.split_documents(docs)

print(f"Loaded {len(chunks)} chunks")

# 첫 번째 청크 내용 확인용
print(f"첫 번째 청크 내용: {chunks[0].page_content[:100]}...")

d:\llm_serving\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading document...
Splitting document into chunks...
Loaded 735 chunks


In [3]:
from langchain_openai import ChatOpenAI
import json

llm = ChatOpenAI(
    base_url="http://localhost:8002/v1",  # llama.cpp text 서버
    api_key="EMPTY",
    temperature=0
)


EXTRACT_PROMPT = """
Extract key entities and their relationships from the following text into a JSON format.
Entities should be concepts, people, or financial terms.
Relationships should describe how they interact.

Output must be ONLY a valid JSON object like this:
{{
  "entities": [{{ "id": "EntityName", "type": "Type" }}],
  "relations": [{{ "source": "Entity1", "target": "Entity2", "type": "Relationship" }}]
}}

Text:
{input_text}
"""

    
# 재귀적으로 실패 시 반으로 나눠서 재시도
def extract_graph(chunk_text, depth=0):
    """실패 시 텍스트를 반으로 나눠서 재시도하는 함수"""
    try:
        # 1. 시도: 현재 텍스트에서 추출
        formatted_prompt = EXTRACT_PROMPT.format(input_text=chunk_text)
        resp = llm.invoke(formatted_prompt, config={"timeout": 45})
        
        content = resp.content.strip()
        
        # --- JSON 정제 로직 시작 ---
        start_idx = content.find('{')
        end_idx = content.rfind('}') + 1
        
        if start_idx == -1 or end_idx == 0:
            raise ValueError("JSON 구조를 찾을 수 없습니다.")
            
        refined_content = content[start_idx:end_idx]
        data = json.loads(refined_content)
        
        # 데이터 구조 보장 (entities, relations 키가 없을 경우 대비)
        if "entities" not in data: data["entities"] = []
        if "relations" not in data: data["relations"] = []
        
        return data
        # --- JSON 정제 로직 끝 ---

    except Exception as e:
        # 너무 많이 쪼갰다면 중단 (최대 깊이 2)
        if depth >= 2: 
            print(f"   ❌ {depth}단계 분할 후에도 추출 실패 (오류: {str(e)[:20]}...)")
            return {"entities": [], "relations": []}
            
        print(f"   ⚠️ 분할 처리 중... (현재 깊이: {depth+1})")
        
        # 2. 텍스트를 반으로 나눠서 각각 재귀 호출
        half = len(chunk_text) // 2
        part1 = extract_graph(chunk_text[:half], depth + 1)
        part2 = extract_graph(chunk_text[half:], depth + 1)
        
        # 결과 합치기
        return {
            "entities": part1["entities"] + part2["entities"],
            "relations": part1["relations"] + part2["relations"]
        }

In [4]:
# Neo4j에 그래프 저장
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    "bolt://localhost:7687",
    auth=("neo4j", "admin123!")
)

try:
    driver.verify_connectivity()
    print("✅ Neo4j 연결 성공!")
except Exception as e:
    print(f"❌ Neo4j 연결 실패: {e}")
    
def save_to_neo4j(tx, entities, relations):
    for e in entities:
        tx.run(
            "MERGE (n:Entity {id:$id, type:$type})",
            id=e["id"], type=e["type"]
        )

    for r in relations:
        tx.run(
            """
            MATCH (a:Entity {id:$src}), (b:Entity {id:$dst})
            MERGE (a)-[:REL {type:$type}]->(b)
            """,
            src=r["source"],
            dst=r["target"],
            type=r["type"]
        )


✅ Neo4j 연결 성공!


In [5]:
import time
import json

print(f"🚀 총 {len(chunks)}개의 청크 처리를 시작합니다.")
start_time = time.time()

for i in range(len(chunks)): 
    chunk = chunks[i]
    chunk_start = time.time()
    
    print(f"🔄 [{i+1}/{len(chunks)}] 처리 중...", end=" ", flush=True)
    
    try:
        # 그래프 추출 (내부에서 타임아웃 처리됨)
        graph = extract_graph(chunk)
        
        entities = graph.get('entities', [])
        relations = graph.get('relations', [])
        
        if entities or relations:
            with driver.session() as session:
                session.execute_write(save_to_neo4j, entities, relations)
            print(f"✅ 완료! (E:{len(entities)}, R:{len(relations)}) - {time.time()-chunk_start:.1f}초")
        else:
            print(f"⚠️ 데이터 없음 - {time.time()-chunk_start:.1f}초")
            
    except Exception as e:
        # 타임아웃 등이 발생하면 여기로 넘어옵니다.
        print(f"❌ 건너뜀 (에러/타임아웃): {str(e)[:30]}")
        time.sleep(1) # 서버 안정화를 위해 잠시 휴식
        continue

print(f"✨ 작업 완료! 총 소요 시간: {time.time() - start_time:.1f}초")

🚀 총 735개의 청크 처리를 시작합니다.
🔄 [1/735] 처리 중... ✅ 완료! (E:3, R:2) - 5.8초
🔄 [2/735] 처리 중... ✅ 완료! (E:3, R:2) - 3.4초
🔄 [3/735] 처리 중... ✅ 완료! (E:3, R:2) - 4.3초
🔄 [4/735] 처리 중... ✅ 완료! (E:4, R:2) - 3.9초
🔄 [5/735] 처리 중... ✅ 완료! (E:5, R:4) - 6.0초
🔄 [6/735] 처리 중... ✅ 완료! (E:9, R:8) - 11.4초
🔄 [7/735] 처리 중... ✅ 완료! (E:7, R:5) - 8.8초
🔄 [8/735] 처리 중... ✅ 완료! (E:8, R:4) - 8.2초
🔄 [9/735] 처리 중... ✅ 완료! (E:6, R:5) - 8.2초
🔄 [10/735] 처리 중... ✅ 완료! (E:4, R:3) - 5.5초
🔄 [11/735] 처리 중... ✅ 완료! (E:6, R:4) - 6.2초
🔄 [12/735] 처리 중... ✅ 완료! (E:2, R:1) - 2.1초
🔄 [13/735] 처리 중... ✅ 완료! (E:4, R:2) - 3.6초
🔄 [14/735] 처리 중... ✅ 완료! (E:3, R:2) - 3.9초
🔄 [15/735] 처리 중... ✅ 완료! (E:3, R:2) - 3.3초
🔄 [16/735] 처리 중... ✅ 완료! (E:4, R:3) - 4.3초
🔄 [17/735] 처리 중... ✅ 완료! (E:10, R:2) - 7.2초
🔄 [18/735] 처리 중... ✅ 완료! (E:8, R:6) - 8.0초
🔄 [19/735] 처리 중... ✅ 완료! (E:5, R:2) - 4.2초
🔄 [20/735] 처리 중... ✅ 완료! (E:5, R:2) - 4.2초
🔄 [21/735] 처리 중... ✅ 완료! (E:3, R:2) - 3.3초
🔄 [22/735] 처리 중... ✅ 완료! (E:2, R:1) - 2.1초
🔄 [23/735] 처리 중... ✅ 완료! (E:2, R:1) -

In [7]:
# Vector DB 구축
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3"
    )

vectorstore = Chroma(
    collection_name="how_to_invest",
    embedding_function=embeddings,
    persist_directory="./chroma"
)

vectorstore.add_documents(chunks)


C:\Users\wind9\AppData\Local\Temp\ipykernel_12264\1303471701.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


['a52b8196-2621-4cfe-a3aa-60e60b5aba11',
 '537a22c3-e5c5-429f-b4df-60edf5f068ba',
 'dc33f750-81f6-4c7e-9ee3-7af662c02bba',
 '28284d79-5dc9-45f2-ad16-df2e5142002b',
 'd0d93b00-312b-4042-8431-9f289e37f44a',
 '76c67afe-9570-47d0-8b8b-d7149ba264db',
 'a4b1fa13-39d5-4b16-993f-7dcd8068a663',
 'eb3ee6a2-113a-4663-b223-7b25f9a2e6a1',
 'c93cfd6a-ed21-42c3-adf6-905880c16d8f',
 '3d769aa9-3031-45b7-a109-565daf38480c',
 '712a8dd1-85b9-43a5-86dd-f4d57022a13e',
 'afaf1efe-55d6-4e0d-b78a-6a61695ebec6',
 '49932f35-2706-4674-ac77-d9f2897263f5',
 '3d9b7103-3800-4a5b-9556-e452e65bc67d',
 'dc16de18-e1c0-4e42-88ac-2ada9c44f5c4',
 '1d584c75-b357-41f1-85db-3fcffb9108f1',
 '3a2af87f-4319-40c6-9a8f-e660a29d68f3',
 '5a55e66f-6af3-48e6-90b3-499bff32ec99',
 '70b7d4c1-c7c5-4cc9-86ef-fef5c10dce87',
 '08e36d81-8116-4b09-be0b-15c13d9455fa',
 '52584660-2672-41d8-8d85-967445aace42',
 'adf12291-281a-464a-a2b4-3fdaa277e6c0',
 '6d6ced58-37dc-4969-8b4e-404fe015a244',
 'cd463f23-ec5b-4048-90f8-89a556a847c0',
 '0b054933-def8-

In [9]:
# Neo4j Retriever
def graph_retrieve(query: str):
    with driver.session() as session:
        result = session.run(
            """
            MATCH (e:Entity)-[r]->(n)
            WHERE e.id CONTAINS $q
            RETURN e.id AS src, r.type AS rel, n.id AS dst
            LIMIT 20
            """,
            q=query
        )
        return [f"{r['src']} -[{r['rel']}]-> {r['dst']}" for r in result]
    
def improved_graph_retrieve(query_entity):
    # 단순히 포함 여부(CONTAINS)가 아니라, 1~2단계 연결된 노드까지 가져오기
    cypher = """
    MATCH (e:__Entity__)-[r:RELATED]-(neighbor)
    WHERE e.id CONTAINS $q
    RETURN e.id + " " + type(r) + " " + neighbor.id AS info
    LIMIT 10
    """
    with driver.session() as session:
        result = session.run(cypher, q=query_entity)
        return [record["info"] for record in result]


In [10]:
# 질의 과정 (Graph + Vector Hybrid)
def hybrid_retrieve(query: str):
    graph_ctx = graph_retrieve(query)
    docs = vectorstore.similarity_search(query, k=4)

    context = "그래프 정보:\n"
    context += "\n".join(graph_ctx)

    context += "\n\n문서 정보:\n"
    for d in docs:
        context += d.page_content + "\n"

    return context


In [11]:
# 최종 응답 생성
def answer(query: str):
    context = hybrid_retrieve(query)

    prompt = f"""
다음 정보를 기반으로 질문에 답하라.

{context}

질문: {query}
"""

    return llm.invoke(prompt).content


In [14]:
# --- 실제 테스트 실행 ---
question = "What are the main risks of investing in railroad bonds?"
# question = "무엇이 채권 투자에 가장 큰 위험 요소인가?"
print(answer(question))

Based on the information provided, there are several main risks associated with investing in railroad bonds:

1. **Quality of Earnings**: The document mentions that it is important for investors to determine if the earnings from a railroad company are legitimate and not the result of neglecting capital improvements. If there are suspicions that earnings are inflated or not properly capitalized, it could indicate potential issues with the company's financial health.

2. **Margin of Security in Equipment**: The document suggests that the safety of railroad bonds in general can be evaluated by considering the margin of security in the equipment itself. This implies that the condition and reliability of the railroad's equipment can be a significant risk factor.

3. **Net Income and Market Confidence**: The document notes that net income varies and that there is diminished confidence in railroad securities. This suggests that the overall financial performance of the railroad company can be 

In [13]:
# --- 실제 테스트 실행 ---
# question = "What are the main risks of investing in railroad bonds?"
question = "무엇이 채권 투자에 가장 큰 위험 요소인가?"
print(answer(question))

기재된 정보를 바탕으로 가장 큰 위험 요소는 채권의 안전성과 신뢰성이 떨어지는 경우입니다. 채권의 안전성과 신뢰성이 떨어진 경우, 채권의 가격은 감소할 가능성이 높습니다. 이는 채권의 변동성과 변동성이 불확실한 경우에 특히 그렇습니다. 따라서 채권 투자에 가장 큰 위험 요소는 채권의 안전성과 신뢰성이 떨어지는 것입니다.
